In [1]:
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.python import keras as kt
# RNN imports
import keras
from keras.models import Model, Sequential
from keras.layers import Input, LSTM, Dense, RNN, SimpleRNN, GRU
from keras.utils import to_categorical


%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

X_test = np.load("X_test.npy")
y_test = np.load("y_test.npy")
person_train_valid = np.load("person_train_valid.npy")
X_train_valid = np.load("X_train_valid.npy")
y_train_valid = np.load("y_train_valid.npy")
person_test = np.load("person_test.npy")

print ('Training/Valid data shape: {}'.format(X_train_valid.shape))
print ('Test data shape: {}'.format(X_test.shape))
print ('Training/Valid target shape: {}'.format(y_train_valid.shape))
print ('Test target shape: {}'.format(y_test.shape))
print ('Person train/valid shape: {}'.format(person_train_valid.shape))
print ('Person test shape: {}'.format(person_test.shape))

Using TensorFlow backend.


Training/Valid data shape: (2115, 25, 1000)
Test data shape: (443, 25, 1000)
Training/Valid target shape: (2115,)
Test target shape: (443,)
Person train/valid shape: (2115, 1)
Person test shape: (443, 1)


In [2]:
# Separate data by subject

# samples from subject 0
idx_p0_train = np.where(person_train_valid == 0)[0]
idx_p0_test = np.where(person_test == 0)[0]

p0_train_X = X_train_valid[idx_p0_train] 
p0_train_y = y_train_valid[idx_p0_train]

p0_test_X = X_test[idx_p0_test]
p0_test_y = y_test[idx_p0_test]

print(p0_train_X.shape)
print(p0_train_y.shape)
print(p0_test_X.shape)
print(p0_test_y.shape)



(237, 25, 1000)
(237,)
(50, 25, 1000)
(50,)


In [3]:
################################
# Sliding window subsampling

# data size (1000, 25)
# sample size (100, 25)
# for loop over each trial
# for loop over each valid index
# append the cut out section to the new dataset
# append the cut out section's label tothe new dataset labels

test_array_x = np.array([[[1,2,3,4,5,6,7,8,9,10]],
                       [[-1,-2,-3,-4,-5,-6,-7,-8,-9,-10]],
                       [[1,2,3,4,5,6,7,8,9,10]]])


test_array_y = np.array([700,701,702])

crop_size = 100 # 3 # 100
original_train_X = p0_train_X  # test_array_x # p0_train_X
original_train_y =  p0_train_y # test_array_y # p0_train_y

N, C, T = original_train_X.shape
print("Original Data:", original_train_X.shape, original_train_y.shape)
#print("X", original_train_X[300:305], "Y", original_train_y[300:305])
cropped_train_X = np.zeros((N*(T-crop_size+1), C, crop_size))
cropped_train_y = np.zeros(N*(T-crop_size+1))
crops_per_sample = T-crop_size+1


for n in np.arange(N):
    crop_count = 0
    for t in np.arange(T-crop_size+1):
        idx = n*crops_per_sample + crop_count
        cropped_train_X[idx] = original_train_X[n, :, t:t+crop_size]
        cropped_train_y[idx] = original_train_y[n]
        crop_count = crop_count + 1
        

print("Cropped Data:", cropped_train_X.shape, cropped_train_y.shape)
#print("X", cropped_train_X[300:305], "Y", cropped_train_y[300:305])

Original Data: (237, 25, 1000) (237,)
Cropped Data: (213537, 25, 100) (213537,)


In [4]:
################################
# rearrange data for Keras modules

##!!!!!! Now train and test data dimensions do not match

# fomat data to (batch_size, timesteps, input_dim)
k_X_train = np.swapaxes(cropped_train_X, 1, 2)
k_X_test = np.swapaxes(p0_test_X, 1, 2)
print("Swapped axes:",k_X_train.shape, k_X_test.shape)
# remove VOG channels (23-25)
k_X_train = k_X_train[: , : , :22 ]
k_X_test = k_X_test[: , : , :22 ]
print("Removed VOG channels:", k_X_train.shape, k_X_test.shape)

# encode output labels
print("Raw labels:", cropped_train_y[0:10])
k_y_train = cropped_train_y - 769
k_y_test = p0_test_y - 769
print("Fixed:", k_y_train[0:10])
k_y_train_categ = to_categorical(k_y_train, 4)
k_y_test_categ = to_categorical(k_y_test, 4)
print("Categorical one-hot encoding:\n",k_y_train_categ[0:3])

Swapped axes: (213537, 100, 25) (50, 1000, 25)
Removed VOG channels: (213537, 100, 22) (50, 1000, 22)
Raw labels: [771. 771. 771. 771. 771. 771. 771. 771. 771. 771.]
Fixed: [2. 2. 2. 2. 2. 2. 2. 2. 2. 2.]
Categorical one-hot encoding:
 [[0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]]


In [5]:
#################### RNN paramters
hids = 100
actv = 'tanh'
dcy = 0.005
reg = 0.1
lrt = 0.00005


#################### RNN training
input_dim = k_X_train.shape[1:]
gru_units_sub = 128

model = Sequential()
#model.add(keras.layers.Conv1D(22, 10, ))
#model.add(keras.layers.BatchNormalization())
model.add(GRU(gru_units_sub, recurrent_dropout=.4, input_shape=input_dim))
model.add(Dense(4, activation="softmax"))
model.compile(loss="categorical_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])

print(model.summary())

history = model.fit(k_X_train, k_y_train_categ, epochs=25, verbose=1, validation_split=0.2, batch_size=32)



#################### RNN loss and accuracy plotting
# Plot training & validation accuracy values
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()
# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_1 (GRU)                  (None, 128)               57984     
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 516       
Total params: 58,500
Trainable params: 58,500
Non-trainable params: 0
_________________________________________________________________
None
Instructions for updating:
Use tf.cast instead.
Train on 170829 samples, validate on 42708 samples
Epoch 1/25
170829/170829 [==============================] - 336s 2ms/step - loss: 1.0012 - acc: 0.5466 - val_loss: 1.4305 - val_acc: 0.4208
Epoch 2/25
 14560/170829 [=>............................] - ETA: 4:51 - loss: 0.7012 - acc: 0.7166

KeyboardInterrupt: 